In [29]:
# Scripts to prep the MSAG CSV export for matching to GIS data

# New columns are appended to the end of the original MSAG

# You will need the list of USPS standard street type abbreviations

In [30]:
# Original column headers in MSAG
# TN_COUNT, LOW_HOUSE_NUMBER, HIGH_HOUSE_NUMBER, ODD_EVEN_INDICATOR,
# STREET_PREFIX_DIRECTIONAL, STREET_NAME, COMMUNITY_NAME, EXCHG,
# COUNTY_ID, STATE, REGION, AGENCY_ID, SR_TANDEM, ESN,  COMMENTS, 
# MODIFIED_BY, MODIFIED_DATE, TYPE


# Columns added to the dataframe 
# ADD_TYPE_msag (COORD or TRAD), ST_NAME_msag, ST_TYPE_msag, 
# ESN_RANGE_msag (true/false), ST_TYPE_usps (provides the suggested USPS street type), 
# ST_TYPE_correct (true/false. checks for match between the original value and the one
# given by USPS dictionary), COORD_FRAG_msag

In [31]:
# REQUIRED

import pandas as pd
import csv
import os


In [32]:
# DATA IMPORTS

# CSV of street type abbreviations and create a dictionary object
# path to the st type csv
stType_path = (r"path\to\uspsFile.csv")


# Create dataframe
usps_types = pd.read_csv(stType_path, low_memory=False)

# Create dictionary from dataframe. The common abbreviations are the keys, the 
# standard abbreviations are the values
stType_dict = usps_types.set_index('COMMON_ABBR').T.to_dict('list')



# MSAG CSV export
 msag_path = (r"path\to\msag.csv")


# Create the dataframe
msag = pd.read_csv(msag_path, low_memory=False)


# ESN values - This can be a simple list. The purpose 
# is to check if the existing ESN values are within range.
# If ESNs need to be corrected, reference the GIS data and ESN 
# boundaries first
esn_path = (r"path\to\esn.csv")


# create the dataframe
esn = pd.read_csv(esn_path, low_memory = False)

# create the list
esn_list = esn['ESN'].values.tolist()


# OPTIONAL - if data is saved in excel file instead of csv
# excelBookPath = (r"path\to\excelFile.xlsx")
# xls = pd.ExcelFile(excelBookPath)

# You must point to the specific sheet. Can create a different dataframe from each 
# sheet. May be useful for combining the ESN list or st type abbreviations in one file
#msag = pd.read_excel(xls, 'name of sheet')


In [33]:
# FUNCTION
# Check if an entry contains coordinates by searching for "N"
# or "W" in the HOUSE_NUMBER 

def checkCoords(addNum):
    # convert to string
    addNum = str(addNum)
    north = "N"
    west = "W"
    # if 'N' or 'W' exists in the address number, return "COORD"
    if north in addNum: 
        message = "COORD"
    elif west in addNum: 
        message = "COORD"
    else: 
        message = "TRAD" # traditional address number
    return message

In [34]:
# FUNCTION

# Note that these functions are configured to split the data as it exists in the Washington County
# MSAG  
# Where there is a column 'STREET_PREFIX_DIRECTIONAL' containing an abbreviated cardinal direction
# followed by the 'STREET_NAME' containing full name with the st type abbreviated. When the street 
# contains coordinate addresses, the first part of the coordinate address is appended to the street 
# name in the same column after several spaces. For example: 
# "TREE TOPS CT                               W0149"

# Returns the MSAG entry's current st type abbreviation
# Splits the street name field at spaces, returns the st type portion
# based on the number of strings 
def getStType(stName):
    # count total spaces in the string
    space_count = stName.count(" ")
    # split the string at every space
    splitString = stName.split()
    if space_count>3:
        stType = splitString[-2]
    else:
        stType = splitString[-1]

    return stType
        

# Returns the MSAG entry's street name separated from the st type and coordinate fragment
# Note that the indices here count from the string's end
def getStName(stName):
    space_count = stName.count(" ")
    splitString = stName.split()
    if space_count>3:
        name = splitString[:-2]
    else: 
        name = splitString[:-1]
        
    newName = ' '.join(name)
    return newName


def coordComponent(add_type, stName):
    if add_type == "COORD":
        coord = getCoordNum(stName)
    else: 
        coord = " " 
    return coord


# Returns the MSAG entry's coordinate address fragment
# Should only be called on fields found to be coordinate addresses
def getCoordNum(stName):
    space_count = stName.count(" ")
    splitString = stName.split()
    coord = splitString[-1]
    return coord

In [35]:
# FUNCTION 
# Supply the matching USPS street abbreviation when possible

def suggestStAbbr(stType, stType_dict):
    if stType in stType_dict: 
        suggest = str(stType_dict[stType][-1]) # cast as string and use indexing to remove pesky brackets
        return suggest
    else: 
        return None

In [36]:
# Add the new columns to the msag dataframe

In [37]:
# New Column
# ADD_TYPE_msag

# Returns indication of traditional or coordinate address based on one of the 
# house numbers

msag['ADD_TYPE_msag'] = msag.apply(lambda x: checkCoords(x['LOW_HOUSE_NUMBER']), axis=1)

In [38]:
# New Column
# ESN_RANGE_msag

# Check if the MSAG entry's ESN exists in the list used by the county

msag['ESN_RANGE_msag'] = msag['ESN'].isin(esn_list)

In [39]:
# New Column
# ST_NAME_msag

# Returns only the street name - no st type or coordinates

msag['ST_NAME_msag'] = msag['STREET_NAME'].apply(getStName)


In [40]:
# New Column
# COORD_FRAG_msag

# Returns only the coordinate component of the street name field

msag['COORD_FRAG_msag'] = msag.apply(lambda x: coordComponent(x['ADD_TYPE_msag'],x['STREET_NAME']), axis=1)

In [41]:
# New Column
# ST_TYPE_msag

# The current st type for the MSAG entry

msag['ST_TYPE_msag'] = msag['STREET_NAME'].apply(getStType)


In [42]:
# New Column
# ST_TYPE_usps

# The street type match from the USPS standard dictionary

msag['ST_TYPE_usps'] = msag.apply(lambda x: suggestStAbbr(x['ST_TYPE_msag'],stType_dict), axis=1)

In [43]:
# New Column
# ST_TYPE_correct

# Check if the suggested st type matches the original - used for reporting on 
# error types and flagging for revision later

msag['ST_TYPE_correct'] = msag['ST_TYPE_msag'] == msag['ST_TYPE_usps']

In [45]:
# Dataframe to CSV 
msag.to_csv(r"path\to\msag_prep.csv")
